In [1]:
%load_ext autoreload
%autoreload 2

from data_helpers.nc21_reader import Nc21Reader



# Conllu tegemine

Features: https://github.com/estnltk/estnltk/blob/4236f2033110d2bf20fc7f565950c0a2170f8573/estnltk/estnltk/taggers/standard/syntax/visl_rows.ipynb#L39

In [14]:
!mkdir conll
!rm conll/nc21*.conll

mkdir: conll: File exists


In [7]:
%%time
# takes ca 2 mins
file_name = 'data/nc21_Fiction.vert'

my_nc21_reader = Nc21Reader(
   file_name = file_name
)

CPU times: user 50 µs, sys: 43 µs, total: 93 µs
Wall time: 110 µs


In [ ]:
filename_template = './conll/nc21_Fiction_%08d.conll'


doc = None
force_doc = False
file_nr = 1

f = open(filename_template % file_nr, 'w')
f.write('# global.columns = ID FORM LEMMA UPOS XPOS FEATS HEAD DEPREL DEPS MISC\n');

count_lines = 0
lines_per_file = 120000

for sentence_id, graph in my_nc21_reader.get_sentences():
    conllu_lines = []
    
    # print(sentence_id)
    #graph.draw_graph()
    
    sentence_text = ' '.join([graph.nodes[n]['form'] for n in sorted(graph.nodes) if n])
    
    if count_lines > lines_per_file:
        f.close()
        file_nr += 1
        f = open(filename_template % file_nr, 'w')
        f.write('# global.columns = ID FORM LEMMA UPOS XPOS FEATS HEAD DEPREL DEPS MISC\n');
        force_doc = True
        count_lines = 0
        
    
    
    if not doc == graph.get_metadata('doc') or force_doc:
        doc = graph.get_metadata('doc')
        conllu_lines.append(f'# newdoc = {doc}')
        force_doc = False
    
    # print(sentence_text)
    conllu_lines.append(f'# text = {sentence_text}')
    for n in sorted(graph.nodes):
        # print(dict(graph.nodes[n]))
        if not n: 
            continue
        id = graph.nodes[n]['id']
        form = graph.nodes[n]['form']
        lemma = graph.nodes[n]['lemma']
        upos = graph.nodes[n]['pos']
        xpos = upos
        feats = []
        case = graph.get_node_case(n)
        inf_form = graph.get_node_inf_form(n)
        pronoun_type = graph.get_node_pronoun_type(n)
        adjective_type = graph.get_node_adjective_type(n)
        negation = graph.get_node_negation(n)
        inf_form = graph.get_node_inf_form(n)
        substantive_type = graph.get_node_substantive_type(n)
        numeral_type = graph.get_node_numeral_type(n)
        number_format = graph.get_node_number_format(n)
        adposition_type = graph.get_node_adposition_type(n)
        conjunction_type = graph.get_node_conjunction_type(n)
        punctuation_type = graph.get_node_punctuation_type(n)
        abbreviation_type = graph.get_node_abbreviation_type(n)
        capitalized = graph.get_node_capitalized(n)
        person = graph.get_node_person(n)
        tense = graph.get_node_tense(n)
        mood = graph.get_node_mood(n)
        voice = graph.get_node_voice(n)
        number = graph.get_node_number(n)
       
        if case:
            feats.append('Case=%s' % case)
        
        if inf_form:
            feats.append('VerbType=%s' % inf_form)
            
        if mood:
            feats.append('Mood=%s' % mood)
        
        if tense:
            feats.append('Tense=%s' % tense)
                
        if voice:
            feats.append('Voice=%s' % voice)
        
        if number:
            feats.append('Number=%s' % number)
        
        if negation:
            feats.append('Polarity=%s' % negation)
            
        if adposition_type:
            feats.append('AdpType=%s' % adposition_type)
            
            
        if len(feats):
            feats = '|'.join(sorted(feats))
        else:
            feats = '_'
        
        head = graph.nodes[n]['head']
        deprel = graph.nodes[n]['deprel']
        deps = '_'
        if graph.nodes[n]['head']:
            deps = 'H_lemma:%s' % graph.nodes[int(graph.nodes[n]['head'])]['lemma']
        
        misc = '_'
        conllu_lines.append(f'{id}\t{form}\t{lemma}\t{upos}\t{xpos}\t{feats}\t{head}\t{deprel}\t{deps}\t{misc}')
    conllu_lines.append('')
    conllu_lines.append('')
    
    count_lines += len(conllu_lines) - 1   
    f.write('\n'.join(conllu_lines))
    

f.close()

data/nc21_Fiction.vert


TSV lines:  92%|█████████▏| 25392128/27588916 [21:17<02:01, 18082.61it/s] 

In [ ]:
graph.get_metadata('doc')

In [ ]:
print('\n'.join(conllu_lines))

In [ ]:
graph.draw_graph()